In [1]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import camera_caliberation_and_correcting_distortion as cc
import sliding_windows_and_curvature as sw
import thresholded_binary_images as th
import prespective_transform as pt
import draw_lines as dl
import lane_line as l
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math

get_ipython().magic('matplotlib inline')

In [2]:
def get_fit_variables(line_name,binary_warped):
    if line_name == "Left":
        line_fit,line_fitx,ploty,line_lane_inds = sw.sliding_windows_left(binary_warped)
    else:
        line_fit,line_fitx,ploty,line_lane_inds = sw.sliding_windows_right(binary_warped)
    return line_fit,line_fitx,ploty,line_lane_inds

In [3]:
def get_fit_variables_repeat(binary_warped,fit):
    return sw.repeated_sliding_windows_line(binary_warped, fit)

In [4]:
def get_line_points(name,fitx,ploty):
    if name == "Left":
        return np.array([np.transpose(np.vstack([fitx, ploty]))])
    else:
        return np.array([np.flipud(np.transpose(np.vstack([fitx, ploty])))])

In [5]:
def set_line(line,fit,fitx,ploty,offset):
    if fitx is not None:
        line.recent_xfitted.append(fitx)
    if len(line.recent_xfitted) > 0:
        line.bestx = np.array(line.recent_xfitted).mean(axis=0)
    if fit is not None and len(fit) > 0:
        line.fits.append(fit)
        line.best_fit = np.array(line.fits).mean(axis=0)
    #difference in fit coefficients between last and new fits
    if fit is not None and len(fit) > 0 and line.current_fit is not None and len(line.current_fit)>0:
        line.diffs = np.subtract(line.current_fit, fit)
    if fit is not None:
        line.current_fit=fit
    if ploty is not None :
        line.ploty = ploty
    if offset is not None:
        xm_per_pix = 3.7/700 
        line.line_base_pos = offset*xm_per_pix
    #distance in meters of vehicle center from the line
    if ploty is not None and fitx is not None and len(fitx)>0 :
        line.radius_of_curvature= sw.curv_meters_line(ploty,fitx)
        line.points = get_line_points(line.name,fitx,ploty)

In [6]:
def distance(l_points, r_points):
    dis = []
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    for l_point in l_points[0]:
        for r_point in r_points[0]:
            
            left_p_x ,left_p_y = l_point[0],l_point[1]
            right_p_x, right_p_y = r_point[0],r_point[1]
            left_p_x*=xm_per_pix
            right_p_x*=xm_per_pix
            left_p_y*=ym_per_pix
            right_p_y*=ym_per_pix
            
            dis.append(math.sqrt((left_p_x - right_p_x)**2 + (left_p_y - right_p_y)**2))
    return min(dis)

In [7]:
def check_distance(lines):
    left,right = lines[0],lines[1]
    diff = 5
    if len(left.points)>0 and len(right.points)>0:
        diff = distance(left.points, right.points)
    return diff < 4

In [8]:
def check_curv(lines):
    left,right = lines[0],lines[1]
    curv_l = left.radius_of_curvature
    curv_r = right.radius_of_curvature
    if curv_l is not None and curv_r is not None and 100<curv_l<10000 and 100<curv_r<10000:
         return True
    return False

In [9]:
def check_parallel(lines):
    left,right = lines[0],lines[1]
    diff = np.subtract(left.current_fit,right.current_fit)
    return (diff[0]+diff[1])<1

In [10]:
def do_sanity(lines):
    return check_curv(lines) and check_parallel(lines) and check_distance(lines)

In [11]:
def detect_line(binary_warped,line,center):
    line_fit,line_fitx,ploty,line_lane_inds = get_fit_variables(line.name,binary_warped)
    if center is not None and line_fitx is not None and len(line_fitx)>0:
        offset = np.mean(line_fitx)-center
        set_line(line,line_fit,line_fitx,ploty,offset)

In [12]:
def sanity_failed(binary_warped,line):
    old_fit = np.add(line.current_fit, line.diffs)
    repeat = get_fit_variables_repeat(binary_warped,old_fit)
    return repeat

In [13]:
def repeat_succeeded(line,repeat,center):
    line_fit,line_fitx,ploty,line_lane_inds = repeat
    if center is not None and line_fitx is not None and len(line_fitx)>0:
        offset = np.mean(line_fitx)-center
        set_line(line,line_fit,line_fitx,ploty,offset)

In [14]:
def clear_fits(line):
    line.recent_xfitted=[]

In [15]:
def set_line_detected(line,detected):
    line.detected = detected

In [16]:
def process_image_lines(undist,binary_warped, lines,M, M_inv):
    height, width = binary_warped.shape
    center = np.int(width/2)

    for line in lines:
        if line.detected == False:
            detect_line(binary_warped,line,center)   
        else:
            repeat = get_fit_variables_repeat(binary_warped,line.best_fit)
            if repeat:
                repeat_succeeded(line,repeat,center) 
        sanity = do_sanity(lines)
        if sanity:
            set_line_detected(line,True)
        else:  
            num_iter = 11
            fits = np.array(line.fits)
            reversed_fits = fits[::-1]
            len_reversed = len(reversed_fits)
            if len_reversed < num_iter:
                detect_line(binary_warped,line,center)
                set_line_detected(line,False)
            else:
                counter =0
                while not sanity and counter in range(num_iter):
                    repeat = get_fit_variables_repeat(binary_warped,reversed_fits[counter])
                    if repeat:
                        repeat_succeeded(line,repeat,center)
                    sanity = do_sanity(lines)
                    counter+=1
        
                if sanity:
                    set_line_detected(line,True)
                else:
                    detect_line(binary_warped,line,center)
                    set_line_detected(line,False)

In [17]:
def get_offset(lines,image):
    left,right = lines[0],lines[1]
    offset = (left.line_base_pos+right.line_base_pos)/2
    return offset

In [18]:
def get_radius(lines):
    max_radius = 10000
    left,right = lines[0],lines[1]
    left_curv = np.round(left.radius_of_curvature,1)
    right_curv = np.round(right.radius_of_curvature,1)
    if left_curv < max_radius and right_curv < max_radius:
        return (left_curv+right_curv)//2
    elif left_curv<max_radius:
        return left_curv
    elif right_curv<max_radius:
        return right_curv
    else:
        return max_radius

In [19]:
def get_direction(offset):
    direction = None
    if offset < 0:
        direction = "left of"
    elif offset == 0:
        direction = "at"
    else:
        direction = "right of"
    direction+=" center"
    return direction

In [20]:
def write_curv_offset(image,lines):
    car_radius = "Curvature is  "+str(get_radius(lines))+'(m)'
    offset = np.round(get_offset(lines,image),2)
    direction = get_direction(offset)
    car_position = "Vehicle is "+str(abs(offset))+'m '+direction
    cv2.putText(image, car_radius, (50, 50 ), cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
    cv2.putText(image, car_position, (50, 100 ), cv2.FONT_HERSHEY_SIMPLEX, 1, 2)

In [21]:
def adjust_gamma(image, gamma=1.0):
    # build a lookup table mapping the pixel values [0, 255] to
    # their adjusted gamma values
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
        for i in np.arange(0, 256)]).astype("uint8")
 
    # apply gamma correction using the lookup table
    return cv2.LUT(image, table)

In [22]:
def equ_image(image):
    img_yuv = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
    img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])
    img_output = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)
    return img_output

In [23]:
def process_images(image):
    height, width,ch = image.shape
    center = np.int(width/2)
    expected_diss = np.int(center/2)
    
    undist = cc.undistort_img(image,objpoints,imgpoints)
    image = adjust_gamma(undist, gamma=4)
    image = equ_image(image)
    
    binary_warped, M, M_inv = pt.prespective_transform(image)
    result = undist
    process_image_lines(undist,binary_warped, lines,M, M_inv)

    left_fitx = np.array(lines[0].recent_xfitted)
    left_reversed_fitx = left_fitx[::-1] 
    right_fitx = np.array(lines[1].recent_xfitted)
    right_reversed_fitx = right_fitx[::-1]
    
    if len(left_reversed_fitx) > 0 and len(right_reversed_fitx)>0:
        result = dl.draw_lines(undist,M_inv,binary_warped,left_reversed_fitx[0],right_reversed_fitx[0],lines[0].ploty)
        write_curv_offset(result,lines)
    
    return result

In [24]:
def process_test_images(image):
    undist = cc.undistort_img(image,objpoints,imgpoints)
    image = adjust_gamma(undist,4)
    image = equ_image(image)
    
    binary_warped, M, M_inv = pt.prespective_transform(image)
    left_fit,right_fit,left_fitx,right_fitx,ploty,left_lane_inds,right_lane_ind = sw.sliding_windows(binary_warped)
    curv = sw.curv_meters(ploty, left_fitx, right_fitx)
    result = dl.draw_lines(undist,M_inv,binary_warped,left_fitx,right_fitx,ploty)
    return result

In [25]:
def pipeline(input_dir,output_dir,lines,objpoints,imgpoints):
    list_img = os.listdir(input_dir)
    for img_name in list_img:
        input_img_path = os.path.join(input_dir,img_name)
        if os.path.isfile(input_img_path):
            img = cv2.imread(input_img_path)
            output_image = process_test_images(img)
            img_output_path = os.path.join(output_dir,img_name)
            cv2.imwrite(img_output_path,output_image)

In [26]:
# test pipeline on test images
input_dir = "test_images/"
output_dir ="output_images/process_test_images/"
lines = [l.Line("Left"),l.Line("Right")]
objpoints,imgpoints = cc.calibrate_camera()
pipeline(input_dir,output_dir,lines,objpoints,imgpoints)

In [27]:
#project video:
objpoints,imgpoints = cc.calibrate_camera()
lines = [l.Line("Left"),l.Line("Right")]
project_output = 'project_output.mp4'
clip1 = VideoFileClip("project_video.mp4")
project_clip = clip1.fl_image(process_images) #NOTE: this function expects color images!!
get_ipython().magic('time project_clip.write_videofile(project_output, audio=False)')

[MoviePy] >>>> Building video project_output.mp4
[MoviePy] Writing video project_output.mp4


100%|█████████▉| 1260/1261 [50:47<00:02,  2.04s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_output.mp4 

CPU times: user 50min 42s, sys: 1min 14s, total: 51min 56s
Wall time: 50min 48s


In [29]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(project_output))

In [30]:
#project video:
objpoints,imgpoints = cc.calibrate_camera()
lines = [l.Line("Left"),l.Line("Right")]
challenge_output = 'challenge_output.mp4'
clip_challenge = VideoFileClip("challenge_video.mp4")
challenge_clip = clip_challenge.fl_image(process_images) #NOTE: this function expects color images!!
get_ipython().magic('time challenge_clip.write_videofile(challenge_output, audio=False)')

[MoviePy] >>>> Building video challenge_output.mp4
[MoviePy] Writing video challenge_output.mp4


100%|██████████| 485/485 [20:36<00:00,  3.28s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: challenge_output.mp4 

CPU times: user 20min 43s, sys: 19.4 s, total: 21min 2s
Wall time: 20min 37s
